In [1]:
import duckdb
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.stats import lognorm
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [6]:
with duckdb.connect("..//data//cdw.db") as conn:
    df = conn.sql("select * from CANDLES").pl()

# this was fixed in the api aggregation
df = df.with_columns(tradetime=pl.from_epoch(pl.col("time"), time_unit="s"),
                     pcnt_change=pl.col('close').pct_change(),
                     log_pcnt_change=(pl.col('close').pct_change().log1p()) * 100)


In [7]:
df.head()

time,low,high,open,close,volume,tradetime,product_id,pcnt_change,log_pcnt_change
f64,f64,f64,f64,f64,f64,datetime[μs],str,f64,f64
1.7161e9,67121.94,67196.89,67128.59,67145.35,22.520218,2024-05-19 02:50:00,"""BTC-USD""",null,null
1.7161e9,67012.85,67175.99,67012.86,67128.58,32.810958,2024-05-19 02:45:00,"""BTC-USD""",-0.00025,-0.024979
1.7161e9,66990.68,67030.3,66990.68,67012.86,2.934036,2024-05-19 02:40:00,"""BTC-USD""",-0.001724,-0.172534
1.7161e9,66953.73,66997.89,66994.21,66993.26,4.950806,2024-05-19 02:35:00,"""BTC-USD""",-0.000292,-0.029252
1.7161e9,66971.94,67072.32,67069.57,66994.21,10.849125,2024-05-19 02:30:00,"""BTC-USD""",0.000014,0.001418


In [9]:
fig = px.histogram(df, x='pcnt_change', title='Distribution of Change in Close for 5 minute intervals', nbins=500)
fig.show()

In [5]:
# Assuming `df` is your DataFrame
fig = make_subplots(rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.02)

# Add histogram for the first column
fig.add_trace(go.Histogram(x=df['pcnt_change'], name='% Returns'), row=1, col=1)

# Add histogram for the second column
fig.add_trace(go.Histogram(x=df['log_pcnt_change'], name='Log % Returns'), row=2, col=1)

fig.update_layout(title='Histograms for Returns and Log Returns')
fig.show()

In [23]:
df.select(pl.col('pcnt_change').skew().alias('skew in return'),
          pl.col('log_pcnt_change').skew().alias('skew in log return'),
          pl.col('log_pcnt_change').std().alias('variance') ** 2)

skew in return,skew in log return,variance
f64,f64,f64
1.403895,1.322202,0.058172


In [7]:
# Filter by the date part of the datetime column
filtered_df = df.filter(pl.col("tradetime") > pl.date(2024, 4, 21))

In [27]:
filtered_df.with_columns(rolling_var = pl.col('close').rolling_var(12))

time,low,high,open,close,volume,tradetime,product_id,pcnt_change,log_pcnt_change,rolling_var
f64,f64,f64,f64,f64,f64,datetime[μs],str,f64,f64,f64
1.7137e9,64516.94,64578.18,64516.94,64541.74,4.906329,2024-04-21 20:50:00,"""BTC-USD""",null,null,null
1.7137e9,64470.3,64564.91,64535.7,64519.67,16.815888,2024-04-21 20:45:00,"""BTC-USD""",-0.000342,-0.034201,null
1.7137e9,64500.01,64562.4,64550.87,64527.83,25.642362,2024-04-21 20:40:00,"""BTC-USD""",0.000126,0.012647,null
1.7137e9,64514.56,64590.4,64565.01,64552.48,14.34653,2024-04-21 20:35:00,"""BTC-USD""",0.000382,0.038193,null
1.7137e9,64555.01,64635.49,64622.28,64566.45,10.765239,2024-04-21 20:30:00,"""BTC-USD""",0.000216,0.021639,null
…,…,…,…,…,…,…,…,…,…,…
1.7137e9,64843.42,64914.26,64876.04,64905.59,11.65284,2024-04-21 00:25:00,"""BTC-USD""",0.001939,0.193684,14688.007879
1.7137e9,64843.21,64928.26,64918.96,64880.45,8.502578,2024-04-21 00:20:00,"""BTC-USD""",-0.000387,-0.038741,12123.62222
1.7137e9,64826.91,64945.43,64918.19,64912.55,9.76855,2024-04-21 00:15:00,"""BTC-USD""",0.000495,0.049463,10098.578345


In [9]:
fig = go.Figure(data=[go.Candlestick(x=filtered_df['tradetime'],
                open=filtered_df['open'],
                high=filtered_df['high'],
                low=filtered_df['low'],
                close=filtered_df['close'])])

fig.show()

In [10]:
fud_idx = pl.read_csv('..\\data\\fear-and-greed-index_20130428_20240420_(Coinmarketcap).csv')

fud_idx = fud_idx.with_columns(
    DateTime = pl.col("DateTime").str.to_datetime("%Y-%m-%d %H:%M:%S")
)

fud_idx.head()

DateTime,Fear & greed index
datetime[μs],f64
2023-06-28 00:00:00,59.13
2023-06-29 00:00:00,59.09
2023-06-30 00:00:00,60.47
2023-07-01 00:00:00,60.78
2023-07-02 00:00:00,60.87


In [11]:
combined_df = df.join(fud_idx, left_on=pl.col('tradetime'), right_on=pl.col('DateTime'), how="left")

In [12]:
combined_df.filter(pl.col('Fear & greed index').is_not_null())

time,low,high,open,close,volume,tradetime,product_id,pcnt_change,log_pcnt_change,Fear & greed index
f64,f64,f64,f64,f64,f64,datetime[μs],str,f64,f64,f64
1.7134e9,61122.8,61415.54,61275.73,61374.01,41.65749,2024-04-18 00:00:00,"""BTC-USD""",0.001669,0.166724,59.36
1.7133e9,63648.52,63854.37,63814.94,63760.2,18.543534,2024-04-17 00:00:00,"""BTC-USD""",0.001093,0.10925,55.3
1.7132e9,63326.95,63510.2,63452.18,63492.82,43.784795,2024-04-16 00:00:00,"""BTC-USD""",0.002188,0.218515,60.31
1.7131e9,65520.23,65782.01,65764.56,65569.98,79.027867,2024-04-15 00:00:00,"""BTC-USD""",-0.000671,-0.067142,60.14
1.7131e9,63825.96,64383.76,64021.36,64280.45,83.447691,2024-04-14 00:00:00,"""BTC-USD""",-0.000529,-0.052879,64.64
1.7130e9,67055.57,67155.13,67148.62,67055.62,33.437637,2024-04-13 00:00:00,"""BTC-USD""",-0.000159,-0.015926,61.13
1.7129e9,70001.35,70155.23,70020.71,70142.29,15.481221,2024-04-12 00:00:00,"""BTC-USD""",-0.000822,-0.082256,68.37


In [13]:
fig = go.Figure(data=[go.Candlestick(x=combined_df['tradetime'],
                open=combined_df['open'],
                high=combined_df['high'],
                low=combined_df['low'],
                close=combined_df['close'])])


fig.show()

In [14]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Line(x=combined_df['tradetime'], y=combined_df['close'], name="Close"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=combined_df['tradetime'], y=combined_df['Fear & greed index'], name="Fear&Greed Idx", mode="markers", marker=dict(size=12, color='orange')),
    secondary_y=True
    )

# Add figure title
fig.update_layout(
    title_text="BTC-USD w/ Fear & Greed Markers"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

c:\Users\grant\Documents\Projects\crypto-datawarehouse\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Line(x=combined_df['tradetime'], y=combined_df['log_pcnt_change'], name="Close"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=combined_df['tradetime'], y=combined_df['Fear & greed index'], name="Fear&Greed Idx", mode="markers", marker=dict(size=12, color='orange')),
    secondary_y=True
    )

# Add figure title
fig.update_layout(
    title_text="BTC-USD w/ Fear & Greed Markers"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="BTC-USD ($)", secondary_y=False)
fig.update_yaxes(title_text="Fear & Greed Idx", secondary_y=True)

fig.show()

In [16]:
from scipy.stats import shapiro

In [17]:
# not a normal distribution
shapiro(df['close'])

ShapiroResult(statistic=0.9225353834240282, pvalue=3.680109820253765e-36)

In [18]:
x=df['log_pcnt_change'].filter(df['log_pcnt_change'].is_not_null())

In [19]:
shapiro(x)

ShapiroResult(statistic=0.8742259021152518, pvalue=2.5785044156224118e-43)